<a href="https://colab.research.google.com/github/priyanka011011/-Restaurant-Description-Generation-Based-on-Customer-Reviews/blob/main/Spelling_correction_and_lemmatization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=187825 sha256=b500f6480276c71eec51a89f55d15a82462539a607922af8dd49db874e5c2719
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy


In [ ]:
from symspellpy.symspellpy import SymSpell, Verbosity
import re
import pandas as pd
import csv
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
nltk.download('omw')
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm_notebook


[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                    "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                    "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                    "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
                    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is",
                    "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are",
                    "you've": "you have"}

In [ ]:
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

In [ ]:
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [ ]:
def spell_checker_eng_word(input_term1):
    max_edit_distance_dictionary = 1
    prefix_length = 7
    sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
    input_term2 = replace_contractions(input_term1)


    if not sym_spell.load_dictionary('frequency_dictionary_en_82_765.txt', term_index=0,
                                    count_index=1):
        print("Dictionary file not found")
        return

    if not sym_spell.load_dictionary('frequency_bigramdictionary_en_243_342.txt', term_index=0,
                                            count_index=1):
        print("Bigram dictionary file not found")
        return

    input_term2=input_term2.replace(',',' ,').replace('.',' .')
    result = sym_spell.word_segmentation(input_term2,ignore_token="[,.']")
    return(result.corrected_string)

In [ ]:
def correct_spell(sent):
    string_without_slang = slang_trans(sent)
    correct_sent = spell_checker_eng_word(string_without_slang)
    return correct_sent

In [ ]:
def slang_trans(input_term4):
    user_string = input_term4.split(" ")
    j = 0
    for i in user_string:
        with open('slang.txt', 'r') as slang_file:
            data = csv.reader(slang_file, delimiter="=")
            i = re.sub('[^a-zA-Z0-9-_.]', '', i)
            for row in data:
                if i.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
        slang_file.close()
        j = j + 1


    return(' '.join(user_string))

In [ ]:
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
    res_words = []
    for word, tag in wn_tagged:
        if tag is None:
            res_words.append(word)
        else:
            res_words.append(lemmatizer.lemmatize(word, tag))
    return " ".join(res_words)


In [ ]:
df = pd.read_csv('/content/restaurant_review.csv').reset_index()

In [ ]:
correct_review = []
reviews = list(df.review)

In [ ]:
num_cores = multiprocessing.cpu_count()
corrected_review = Parallel(n_jobs=num_cores)(delayed(correct_spell)(reviews) for reviews in tqdm_notebook(reviews))

<ipython-input-60-cab0dbd3e8a3>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  corrected_review = Parallel(n_jobs=num_cores)(delayed(correct_spell)(reviews) for reviews in tqdm_notebook(reviews))


  0%|          | 0/9375 [00:00<?, ?it/s]

In [ ]:
df['corrected_review'] = corrected_review

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm_notebook

def lemmatize_sentence_threadsafe(sentence):
    try:
        # Perform lemmatization here
        # Replace the following line with your lemmatization logic
        lemmatized_sentence = sentence.lower()
        return lemmatized_sentence
    except Exception as e:
        print(f"Error occurred during lemmatization: {e}")
        return None

# Define the number of threads to use for parallel processing
num_threads = multiprocessing.cpu_count()

# Use ThreadPoolExecutor to perform parallel processing
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    lemmatised_review = list(tqdm_notebook(executor.map(lemmatize_sentence_threadsafe, corrected_review), total=len(corrected_review)))

# Now, store the results in the DataFrame
df['cor_lem_review'] = lemmatised_review


  0%|          | 0/9375 [00:00<?, ?it/s]

In [ ]:
df.to_csv('Restaurant_Corrected_Review.csv')

In [ ]:
df.to_csv('Restaurant_Correct_Review.csv',index=False)

In [ ]:
file_path = '/content/Restaurant_Corrected_Review.csv'

df.to_csv(file_path, index=False)

In [ ]:
from google.colab import files

files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>